In [ ]:
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import netCDF4 as nc
from netCDF4 import Dataset
import math
from IPython.display import HTML

In [ ]:
#How to see the groups in the nc file
# Full path to your NetCDF file
file_path = '/Users/.../.../results.nc'

# Get the group names
group_names = Dataset(file_path).groups.keys()
print("Groups in file:", list(group_names))

# Load each group into an xarray dataset
datasets = {}
for group_name in group_names:
    datasets[group_name] = xr.open_dataset(file_path, group=group_name)

# Example: How to access the 'totcon' group
display(datasets["volume"])

In [ ]:
#Plotting different runs at a fixed time
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
subsetds_volume = datasets['volume'].sel(time = 2.5, Y= 5, Z = 5,method='nearest')
subsetds_pH = datasets['pH'].sel(time = 2.5, Y= 5, Z = 5,method='nearest')
subsetds_volume['Calcite'].plot.line(y='X', yincrease=False,ax=axs[0])
subsetds_pH['pH'].plot.line(y='X', yincrease=False,ax=axs[1])


In [ ]:
#Plotting all runs at a each time
datasets['totcon'].sel(Y=5, Z=5)['SO4--'].plot.line(y='X',col='time', col_wrap=4)

In [ ]:
#Plot all minerals

# Extract the dataset
ds = datasets['volume']

# Set fixed indices
file_idx = 1  # or 0 if needed
y_idx = 0
z_idx = 0

# Get variable names
var_names = list(ds.data_vars.keys())
n_vars = len(var_names)

# Define subplot layout (e.g., 3 columns)
cols = 3
rows = math.ceil(n_vars / cols)

# Create figure and axes
fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), constrained_layout=True)

# Flatten axes for easy iteration
axes = axes.flatten()

# Plot each variable
for i, var_name in enumerate(var_names):
    ax = axes[i]
    data = ds[var_name].isel(file_num=file_idx, Y=y_idx, Z=z_idx)
    
    data.plot(ax=ax, x="X", y="time", cmap="viridis", add_colorbar=False)
    ax.set_title(var_name)
    ax.set_xlabel("X")
    ax.set_ylabel("Time")

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Optional: show or save the plot
plt.suptitle("Mineral Volume Evolution", fontsize=16)
plt.show()

In [ ]:
#Plot fluid evolution

# Extract the dataset
ds = datasets['totcon']

# Set fixed indices
file_idx = 14
y_idx = 0
z_idx = 0

# Get variable names
var_names = list(ds.data_vars.keys())
n_vars = len(var_names)

# Define subplot layout (e.g., 3 columns)
cols = 3
rows = math.ceil(n_vars / cols)

# Create figure and axes
fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), constrained_layout=True)

# Flatten axes for easy iteration
axes = axes.flatten()

# Plot each variable
for i, var_name in enumerate(var_names):
    ax = axes[i]
    data = ds[var_name].isel(file_num=file_idx, Y=y_idx, Z=z_idx)
    
    data.plot(ax=ax, x="X", y="time", cmap="viridis", add_colorbar=False)
    ax.set_title(var_name)
    ax.set_xlabel("X")
    ax.set_ylabel("Time")

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Optional: show or save the plot
plt.suptitle("Fluid Evolution", fontsize=16)
plt.show()

In [ ]:
ds = datasets['volume']  # Your dataset
var_names = list(ds.data_vars)
n_vars = len(var_names)

cols = 3
rows = math.ceil(n_vars / cols)

fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), constrained_layout=True)
axes = axes.flatten()

# Fix dimension length access
num_files = ds.sizes['file_num']
y_idx = 0
z_idx = 0

def update(file_idx):
    for i, var_name in enumerate(var_names):
        ax = axes[i]
        ax.clear()
        
        data = ds[var_name].isel(file_num=file_idx, Y=y_idx, Z=z_idx)
        data.plot(ax=ax, x="X", y="time", cmap="viridis", add_colorbar=False)
        ax.set_title(var_name)
        ax.set_xlabel("X")
        ax.set_ylabel("Time")

    # Hide unused axes
    for j in range(i + 1, len(axes)):
        axes[j].clear()
        axes[j].axis("off")

    fig.suptitle(f"file_num = {file_idx}", fontsize=16)

ani = animation.FuncAnimation(fig, update, frames=range(num_files), interval=1000, repeat=False)

HTML(ani.to_html5_video())

In [ ]:
ds = datasets['volume']  # Replace with your group
var_names = list(ds.data_vars)

file_idx = 0
y_idx = 0
z_idx = 0

time_vals = ds.coords['time'].values
x_vals = ds.coords['X'].values
n_vars = len(var_names)

# --- Plot grid layout ---
cols = 3
rows = math.ceil(n_vars / cols)

fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), constrained_layout=True)
axes = axes.flatten()

# --- Animation update function ---
def update(frame):
    fig.suptitle(f"Time = {time_vals[frame]:.3f}", fontsize=16)

    for i, var in enumerate(var_names):
        ax = axes[i]
        ax.clear()

        y = ds[var].isel(file_num=file_idx, Y=y_idx, Z=z_idx, time=frame)
        ax.plot(x_vals, y, label=var, color='royalblue')
        ax.set_title(var)
        ax.set_xlabel("X")
        ax.set_ylabel(var)
        ax.grid(True)

    # Hide any unused axes
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

# --- Animate ---
ani = animation.FuncAnimation(
    fig, update, frames=len(time_vals), interval=800, repeat=False
)

HTML(ani.to_html5_video())